# ARE Forêt Sombre Suivi Code

## Introduction

Ce notebook a été créé pour montrer l'évolution et les stratégies adoptées dans la conception de la simulation que nous avons dévellopée.

# ATTENTION

Ce notebook est à titre descriptif, le code est souvent incomplet car seules les modifications sont indiquées. L'intégralité des différentes versions est retrouvable sur le <link><a href="https://github.com/are-dynamic-2021-g5/foret-sombre/blob/main/MesaModel">github</a></link> ou depuis le <link><a href="https://areforetsombre.wixsite.com/foretsombre/code">site internet</a></link> (cliquez sur Github).

Par conséquent, rien ne sert de Run les cells.

### Importer les librairies nécessaire à l'éxécution du code 

In [2]:
#import sys
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install mesa

Le code ci-dessus devrait faire l'affaire mais comme je n'ai absolument aucune idée de comment ce petit 
!{sys.executable} fonctionne et que je n'ai pas envie de me faire taper sur les doigts, je vous prierai d'installer ces deux librairies comme vous avez l'habitude de faire.
Avec par exemple :

`pip install matlpotlib` <br>
`pip install mes` <br>
<br>
ou (si ça ne fonctionne pas) un truc du genre : <br>
<br>
`python3 -m pip install xxxxx` <br>
`pip3 install xxxx` <br>

### Premier test avec pygame

command d'install : pip install pygame

La première idée venue a été celle d'utiliser la library moteur de jeu qui permet d'accéder simplement à une interface graphique. C'est à l'origine un moteur de jeu-vidéo qui permet notament de simuler facilement des intéractions physique : collision, mouvement sur un plan 2D et 3D etc...

Dans un premier temps, afin de découvrir les différentes possibilitées que cette librairy apporte j'ai décidé de créer différents type de signaux qui pourrait servir à simuler la communication inter-civilisations. 

Deux types de signaux sont conceptualisés : 
- Multidirectionnels rectiligne uniforme (en cercle)
- Multidirectionnels rectiligne individualisés (cercle en nuage de points indépendants)

##### En "cercle"

<li><a href="https://github.com/are-dynamic-2021-g5/foret-sombre/blob/main/_Old_PygameModel/signal-cercle.py">Lien vers le github</a></li>

##### En "Nuage de points"

<li><a href="https://github.com/are-dynamic-2021-g5/foret-sombre/blob/main/_Old_PygameModel/signal-nuage-point.py">Lien vers le github</a></li>

#### Remarque

Le code est long, j'ai préféré mettre les liens à la place.

Pygame n'est absolument pas le module adapté pour ce que nous cherchons à faire, bien trop orientés moteur de jeu/physique qu'il ne le faut pour ce projet.

### Second test avec Mesa ABM (agent base modelling)

Mesa ABM, une libraire python basée sur les modèles à base d'agent semble beaucoup plus adapté à une simulation simple et efficace de la théorie de la forêt sombre. Elle permet de faire évoluer les différents agents, dans notre cas, les différentes civilisations en fonction d'une timeline (Model.schedule) et offre une grande simplicité de visualisation avec une page web sur laquelle s'affiche nos données.

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

import random


class CivAgent(Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.ems_ablt = random.randrange(10)  # Capacité à émettre des signaux
        # Capacité à recevoir des signaux
        self.rcpt_ablt = random.randrange(10)
        # Type de civilisation : Pacifique ou Aggressive
        self.type = bool(random.getrandbits(1))
        self.tech_lvl = random.randrange(10)  # Niveau technologique

    def step(self):
        print("Agent {} initialized".format(self.unique_id),
              "Emission ability : {}".format(self.ems_ablt),
              "Reception ability : {}".format(self.rcpt_ablt),
              "Tech lvl : {}".format(self.tech_lvl),
              "Type : {}".format(self.type))


class CivModel(Model):

    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)  # Créer une timeline

        for i in range(self.num_agents):

            agent = CivAgent(i, self)
            self.schedule.add(agent)  # ajoute N agent à la timeline
            # positionne aléatoirement l'agent sur la grille
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

    def step(self):
        self.schedule.step()

L'"environnement intéractif" (le main()) :

In [ ]:
from DfModel import CivModel

empty_model = CivModel(11)
empty_model.step()

Pour visualiser :

In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

from DfModel import CivModel

# Grid design
def agent_portrayal(agent):
	portrayal = {"Shape": "circle",
				 "Color": "red", 
				 "Filled": "true",
				 "Layer": 0,
				 "r": 0.5}
	return portrayal 

# Grid dimension
WIDTH, HEIGHT = 500, 500

# Grid squaring
nb_square_x, nb_square_y = 10, 10
grid = CanvasGrid(agent_portrayal, nb_square_x, nb_square_y, WIDTH, HEIGHT) # Initialize grid


server = ModularServer(CivModel, # Initialize Server for Model "CivModel"
					   [grid], 	 # Print grids
					   "Dark Forest Model", # Title
					   {"N":5, "width":10, "height":10}) # Declare CivModel arguments

server.port = 8521 # Set port
server.launch() 

Le résultat n'est pas très excitant, mais il démontre bien ce qu'il est possible de faire en 50 lignes.

#### version 1

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

import random
from collections import OrderedDict


class CivAgent(Agent):
    """ An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.emission = random.randrange(2)  # Capacité à émettre des signaux
        # Capacité à recevoir des signaux
        self.reception = random.randrange(2)
        # Type de civilisation : Pacifique ou Aggressive
        self.type = bool(random.getrandbits(1))
        self.tech_lvl = random.randrange(10)  # Niveau technologique
        # Positions des agents qui ne sont déclarés qu'après les dimensions de la grid
        self.x, self.y = None, None
        self.pos = None

    def step(self):
        print("Agent", self.unique_id, " initialized",
              #"Emission ability : {}".format(self.emission),
              #"Reception ability : {}".format(self.reception),
              #"Tech lvl : {}".format(self.tech_lvl),
              "Type :", "Aggressive" if self.type else "Pacifique")


class CivModel(Model):
    """A model with some number of agents."""

    def __init__(self, N, width=500, height=500):
        self.nb_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)  # Créer une timeline

        for i in range(self.nb_agents):

            agent = CivAgent(i, self)
            self.schedule.add(agent)  # ajoute N agent à la timeline
            # positionne aléatoirement l'agent sur la grille
            agent.x = self.random.randrange(self.grid.width)
            agent.y = self.random.randrange(self.grid.height)
            agent.pos = agent.x, agent.y
            self.grid.place_agent(agent, agent.pos)

    def random_connect(self):
        """Renvoie une liste de pair d'agent, dans l'ordre si random=False"""
        random_id = random.sample(
            [k for k in range(self.nb_agents)], self.nb_agents)
        n_l = [k for k in range(self.nb_agents)]
        connection = list(zip(n_l, random_id))
        return connection

    def contact(self):
        """Première aspect de la logique utilisé, random_connect crée une 
            liste de tuple qui relie deux CivAgent entre eux cette fonction compare leur 
            self.type (0 pour Pacifique (P) et 1 pour aggressif (A)) et leur niveau technologique
            (self.tech_lvl) si besoin."""

        #print(self.schedule._agents[2])
        #print(list(self.schedule._agents))
        r_l = self.random_connect()
        #print(r_l)
        for a, b in r_l:
            #print(a)

            # Si l'agent a deja ete remove, continue
            if a in list(self.schedule._agents) and b in list(self.schedule._agents):
                agent_a = self.schedule._agents[a]
                agent_b = self.schedule._agents[b]
            else:
                continue

            if agent_a.type < agent_b.type:  # Si b A et a P
                self.schedule.remove(agent_a)  # remove a
                print("Agent", a, "destroyed by Agent", b)
            elif agent_a.type > agent_b.type:  # Si b P et a A
                self.schedule.remove(agent_b)  # remove b
                print("Agent", b, "destroyed by Agent", a)
            elif agent_a.type == 1 and agent_b.type == 1:  # Si b A et a A, regarde le tech_lvl
                print("Agents", a, "and", b, "are both aggresive")
                if agent_a.tech_lvl == agent_b.tech_lvl:  # btech == atech <=> b P et a P
                    print(a, "and", b, "have the same technological level, nothing appends")
                    continue
                elif agent_a.tech_lvl < agent_b.tech_lvl:
                    self.schedule.remove(agent_a)
                    print("Agent", b ,"stronger than Agent", a)
                    print("Agent", a, "destroyed by Agent", b)
                else:
                    self.schedule.remove(agent_b)
                    print("Agent", a ,"stronger than Agent", b)
                    print("Agent", b, "destroyed by Agent", a)
            else:
                continue
        # Renvoie la liste des agents ayant survécu
        survivors = [index for index in list(self.schedule._agents)]
        print("Survivors:", survivors)
        return survivors

    def step(self):
        self.schedule.step()
        self.contact()

def main():
    empty_model = CivModel(11)
    for i in range(5):
        empty_model.step()

if __name__ == "__main__":
    main()

Premier aspect dynamique ! Ici on se passe totalement de l'espace 2D (qui n'arrivera pas tout de suite) mais on obtient déjà un semblant de résultat.

#### Version 2

J'ai implémenté un nouveau système pour simuler le repérage des diverses civilisations. J'ai mis de côté (pour l'instant et, qui sait, peut être pour toujours) la dimension spatial du model, qui, car trop complexe et non nécessaire à l'obtention de résultat concret, n'a pour l'instant pas lieu d'être.
À la place on caractérise chaque civilisation par des paramètres `emission` et  `reception`. Chaque civilisations (souvent appelées agents dans les commentaires du codes) ont un niveau d'émission et de réception donné tel que si une civilisation à un petit (resp. grand) niveau d'émission, elle sera plus difficile (resp.plus facile) à repérer. De même, la capacité de réception d'une civilisation représente sa capacité à recevoir/interpréter des signaux. L'implémentation de ces deux paramètres fonctionnent comme suis : 

Plus le niveau d'emission est haut, moins un niveau de réception haut est nécessaire pour entrer en contact (du moins dans un sens, pour l'instant) à l'inverse plus le niveau d'émission est bas, plus la difficulté à repérer une telle civilisation est grande et donc plus un niveau élevé de reception est nécessaire.

On pourrait vouloir associer directement le niveau technologique d'une civilisation extra-terrestre avec ses niveaux d'émissions et de réceptions, néanmoins, je mettrai de côté cet idée car il est plausible que le niveau d'avancement d'une civilisations ne soit pas la cause d'un niveau d'émission élevé (Cf. Qualitative classification of extraterrestrial civilizations) ou que par choix - comme le suppose Liu Cixin (Cf. The Dark Forest Theory (1)) - choissisent de ne rien émettre ou de ne rien recevoir.

Le niveau technologique sera donc utilisé pour résoudre la logique du conflit, une technologie plus avancée donnant un avantage certain.

In [ ]:
def detect(self, agentA, agentB, both=True):
    """Renvoie si oui ou non l'agent a detect l'agent b
        Fonctionnement du système émission/réception:
        Plus un agent émet de signaux, plus il est repérable. Le niveau de reception
        définit la capacité d'un agent à détecter des signaux.
        Par conséquent plus le niveau d'emission d'un agent est haut plus il
        est facilement reperable par des agents dont le niveau de reception est bas.
        """
    if agentA.reception == 0:
        return False
        print("Agent", self.schedule._agents, "ne peut rien reperer") # Cas ou reception = 0, L'agent ne peut rien reperer
    else:
        # Créé une liste par compréhension dont les élement sont les niveaux d'emissions reperables par l'agent A, 
        # si l'agent B a un niveau d'emission appartement à cette liste alors A peut reperer B
        spotable_agents = [k for k in range(emission_range-1, emission_range-agentA.reception-1, -1)]
        print("Agent", agentA.unique_id, "R"+str(agentA.reception), "Agent", agentB.unique_id, "E"+str(agentB.emission), "l'agent peut reperer ceux dont l'emission est : ", spotable_agents)
        return agentB.emission in spotable_agents # renvoie vrai si l'agentB peut etre repere par A ou faux sinon

En plus de l'implémentation des paramètres reception/emission, les logs sont plus claires, on peut déjà entrevoir la validité de plusieurs hypothèses en modifiant les différents paramètres.

##### Version 3

Dans cette troisième version j'ai implémenté la distance entre les agents, pour l'instant en deux dimensions pour des raisons de rapidité mais il sera très facile de passer à un espace tridimensionnel. J'ai aussi amélioré l'algorythme (maintenant plus rapide) permettant de déterminer si une civilisation détecte/est détectable par rapport à son niveau d'émission/réception. 

Fonctionnement de la prise en compte de la distance :

À la manière de Kent A. Peacock avec la loi de Lotka (Cf. Fermi and Lotka: The Long Odds of Survival in a Dangerous Universe (3)), on utilise la fonction exponentielle pour modéliser la difficulté qu'ont les hypothétiques civilisations à communiquer, c'est à dire les différents types d'interférences, les signaux perdus etc... 
Pour ce faire on utilise la fonction : exp(-kx) où x est la distance séparant deux civilisations et k>0 le facteur "d'opacité" de l'univers, plus celui-ci est élevé plus il est difficile, même avec de très bon niveau d'émission et de réception, d'intercepter des signaux compréhensible d'autres intelligences extra-terrestrielles.
C'est une manière simple est assez efficace de représenter les différents aléas de l'immensité cosmique qui interviennent dans la tentative de communication (Cf. BRIN, G. D. The Great Silence (1)).

In [ ]:
univers_scale = 1000
opacity_factor = 1
threshold = 0.001
# random.seed(0)

class CivAgent(Agent):
    """ An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        self.x, self.y = random.randrange(1000), random.randrange(1000)
        self.pos = self.x, self.y

    def step(self):
        print("Agent", self.unique_id, " initialized",
              "Emission ability : ", self.emission,
              "Reception ability : ", self.reception,
              "Tech lvl : ", self.tech_lvl,
              "Type :", "Aggressive" if self.type else "Pacifique")


class CivModel(Model):
    """A model with some number of agents."""

    def __init__(self, N):
        self.distances_log = self.calculate_distance()

    def calculate_distance(self):
        distances_log = {}
        for i in self.schedule._agents:
            #print(distances_log, len(distances_log))
            if i in self.schedule._agents.keys():
                agentA = self.schedule._agents[i]
                for j in range(i+1, len(self.schedule._agents)):
                    if j in self.schedule._agents.keys():
                        agentB = self.schedule._agents[j]
                        d = math.sqrt((agentA.x-agentB.x)**2+(agentA.y-agentB.y)**2)
                        distances_log[(i, j)] = d
        return distances_log

    def detect_distance_check(self, distance, opacity_factor, threshold):
        d_scaled = distance/univers_scale
        return math.exp(-opacity_factor*d_scaled)

    def detect(self, agentA, agentB, both=True):
        """Renvoie si oui ou non l'agent a detect l'agent b
            Fonctionnement du système émission/réception:
            Plus un agent émet de signaux, plus il est repérable. Le niveau de reception
            définit la capacité d'un agent à détecter des signaux.
            Par conséquent plus le niveau d'emission d'un agent est haut plus il
            est facilement reperable par des agents dont le niveau de reception est bas.
            """
        if agentA.reception == 0:
            return False
            # Cas ou reception = 0, L'agent ne peut rien reperer
            print("Agent", self.schedule._agents, "ne peut rien reperer")
        else:
            a = self.detect_distance_check(self.distances_log[(min(agentA.unique_id, agentB.unique_id), max(agentA.unique_id, agentB.unique_id))], opacity_factor, threshold)
            print(self.distances_log[(min(agentA.unique_id, agentB.unique_id), max(agentA.unique_id, agentB.unique_id))], a, threshold)
            if not a >= threshold:
                print("Trop loin...")
                
            return agentA.reception + agentB.emission >= reception_range+1 and a >= threshold# max(R) + 1

##### Version 4

Dans cette version on a implémenté plusieurs nouveaux éléments. Dans un premier temps une (très légère) interface visuelle a été ajouté : on peut désormais voir qu'elles civilisations on résisté en fonction de leur position dans un espace 2D. Ensuite, on a ajouté des "clusters", la matière dans l'univers n'étant pas répartit uniformément, mais par petites zone éloignée les unes de autres, on a implémenté des zones de rayon restreint dans lesquels sont placées les civilisations à l'initialisation du modèl (voir photo + source). Un nouveau mode de connection à aussi été implémenté, le mode "nearest neighbors" dans lequel les connections ne sont pas faites aléatoirement comme dans les versions antérieures mais avec le voisins le plus proches. Pour ce faire on a calculé et classé par ordre décroissant l'intégralité des distances entre les agents. La composante z a aussi été ajouté. Petit point sur les résultats.

In [ ]:
from tkinter import *

##### TKINTER PARAMETERS & INITIALIZATION #####
H = 500
W = 500
window = Tk()
canvas = Canvas(window, width=W, height=H, bg='black')
canvas.pack()
###############################################

##### IMPORTANT PARAMETERS #####
# AGENT PARAMETERS
emission_range = 5
reception_range = 5
technological_range = 10

# UNIVERS PARAMETER
univers_scale = 500
opacity_factor = 1
threshold = 0.001
nb_clusters = 3
clusters_scale = 100
################################
# random.seed(0)

class CivAgent(Agent):

    def __init__(self, unique_id, model):
        # Coordinates
        self.x = None
        self.y = None
        self.z = None  # for 3D

    def step(self):

        print("Agent", self.unique_id, " initialized",
              "Emission ability : ", self.emission,
              "Reception ability : ", self.reception,
              "Tech lvl : ", self.tech_lvl,
              "Type :", "Aggressive" if self.type else "Pacifique")

class CivModel(Model):

    def __init__(self, N):
        # Initializations spatials des agents
        positions = self.spawn_clusters()
        #print(positions, len(positions))

        for i in range(self.nb_agents):
            # positionne aléatoirement l'agent sur la grille
            agent = CivAgent(i, self)
            agent.x, agent.y, agent.z = positions[i]
            self.schedule.add(agent)  # ajoute les agents à la timeline

        # Calcules les distances entre les agents
        self.distances_log = self.calculate_distance()

    def spawn_clusters(self):
        """Répartis les civilisations en clusters de densité égale 
           dans l'univers (peut être assimiler à des galaxies)"""
        coords = []

        for i in range(nb_clusters):
            clx, cly, clz = random.randint(0, univers_scale), random.randint(
                0, univers_scale), random.randint(0, univers_scale)  # Créer un point de cluster

            # Créé un nombres de groupe d'agent équivalent à celui des clusters
            rd_nb_civ = int(self.nb_agents/nb_clusters)+1
            # Répartis aléatoiremnt les civilisations autour de ce clusters dans un radius donné.
            for j in range(rd_nb_civ):
                r = clusters_scale
                pl_x, pl_y, pl_z = random.randint(max(
                    0, clx-r), min(univers_scale, clx+r)), random.randint(max(0, cly-r), min(univers_scale, cly+r)), random.randint(max(0, clz-r), min(univers_scale, clz+r))
                coords.append((pl_x, pl_y, pl_z))
        return coords

    def sort_dict(self, dic, first_term=False):
        sorted_dict = {}
        if first_term:
            for i in range(self.nb_agents):
                sorted_dict.update({k: v for k,
                                    v in sorted(dic.items(), key=lambda item: item[1]) if i is k[0]})
        else:
            sorted_dict = {k: v for k, v in sorted(
                dic.items(), key=lambda item: item[1])}
        return sorted_dict

    def nn_connect(self):
        """Connect chaque agent avec les autres en partant du plus proches"""
        distances_dict = self.calculate_distance()
        for i in range(self.nb_agents):
            order_of_connection = self.sort_dict(distances_dict)
        # print(order_of_connection)
        return order_of_connection.keys()

    def calculate_distance(self):
        """Créé un dictionnaire dont les keys sont la pair d'agent en question la la value la distance les séparant"""
        distances_log = {}
        for i in self.schedule._agents:
            #print(distances_log, len(distances_log))
            if i in self.schedule._agents.keys():
                agentA = self.schedule._agents[i]
                for j in range(i+1, len(self.schedule._agents)):
                    if j in self.schedule._agents.keys():
                        agentB = self.schedule._agents[j]
                        d = math.sqrt((agentA.x-agentB.x)**2 +
                                      (agentA.y-agentB.y)**2 + (agentA.z-agentB.z)**2)
                        distances_log[(i, j)] = d
        return distances_log
    
def main():
    N = 100
    model = CivModel(N)
    # print(model.nn_connect())
    model.step()
    draw(canvas, model.schedule._agents, 'white')

    for i in range(1000):
        print("Tour : ", i)
        model.step()
        print("Nombre d'agents restant", len(model.schedule._agents))

    draw(canvas, model.schedule._agents, 'green')
    model.schedule.step()
    window.mainloop()  # Visualize!


def draw(canvas, agents_list, color):
    for agent in agents_list.values():
        canvas.create_oval(agent.x, agent.y, agent.x +
                           10, agent.y+10, fill=color)

#### Version 5

Dans cette 5e version, j'ai ajouté l'équation de Drake afin d'avoir un nombre de civilisation en rapport avec des facteurs intéressant, qui crédibilise notre modèle. Ils sont décris ci-dessous. Dans son article _The Great Silence_ Brin G. D. ajoute deux facteur supplémentaire à cette équation, ces facteurs influent surtout sur la probabilité d'une rencontre humain/extraterrestre, il ne m'a pas paru pertinent de les ajouter.
(Cf. le site onglet 'Annexe' pour la définitions complète des équations de Drake)

Matplotlib a aussi été implémenté afin d'avoir un meilleur aperçu de ce qu'il passe au sein du modèle, vitesse de décroissance de la démographie galactique, ratio de civilisation aggressive encore en vie etc..

In [ ]:
import matplotlib.pyplot as plt

###############################################

##### IMPORTANT PARAMETERS #####

# DRAKE EQUATION PARAMETERS : E = R*fg*ne*fl*fi*fc*L
R = 1 # 1 - 3 # average galactic rate of star production
fg = 0.1 # 0.1 #fraction of 'good' and stable dwarves accompanied by planets
ne = 0.1 # 0.1 - 5 # number of candidates planets per system 
fl = 0.1 # 0.1 - 1.0 / 10e-8 # fraction of said planets upon which life arises 
fi = 0.1 # 0.01 - 1.0 / 10e-8 # fraction of the latter upon which intelligence evolves
fc = 0.5 # 0.01 - 1.0 # fraction of intelligent species which devellop detectable tech
L = 10**6 # arbitrary # the average lifespan of such a technological culture
global E 
E = round(R*fg*ne*fl*fi*fc*L) # the expected number of populated site in a galaxy

################################

############ Plotting Graphs ##################################################

    def plot_agents_positions(self, color):
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')

        for agent in self.schedule._agents.values():
            ax.scatter(agent.x, agent.y, agent.z, color=color, marker='o')

        plt.show()

    def plot_agents_type(self):
        fig, axs = plt.subplots(1, 1, figsize=(5, 5)) 
        types = [int(a.type) for a in self.schedule._agents.values()]
        
        return axs.hist(types, bins=2)

    def plot_agents_tech(self):
        fig, axs = plt.subplots(1, 1, figsize=(5, 5)) 
        tech = [a.tech_lvl for a in self.schedule._agents.values()]
        
        return axs.hist(tech, bins=technological_range)

    def plot_agents_ems(self):
        fig, axs = plt.subplots(1, 1, figsize=(5, 5)) 
        ems = [a.emission for a in self.schedule._agents.values()]
        
        return axs.hist(ems, bins=emission_range)

    def plot_agents_rec(self):
        fig, axs = plt.subplots(1, 1, figsize=(5, 5)) 
        rec = [a.reception for a in self.schedule._agents.values()]
        
        return axs.hist(rec, bins=reception_range)

###############################################################################

def main():
    model = CivModel(E)
    #print(model.nn_connect())
    model.step()
    #draw(canvas, model.schedule._agents, 'white')
    model.plot_agents_positions('b')
    model.plot_agents_type()
    model.plot_agents_tech()
    model.plot_agents_ems()
    model.plot_agents_rec()
    plt.show()
    for i in range(100):
        print("Tour : ", i)
        model.step()
        print("Nombre d'agents restant", len(model.schedule._agents))

    #draw(canvas, model.schedule._agents, 'green')
    #window.mainloop()  # Visualize!


def draw(canvas, agents_list, color):
    for agent in agents_list.values():
        canvas.create_oval(agent.x, agent.y, agent.x +
                           10, agent.y+10, fill=color)

if __name__ == "__main__":
    main()

#### Version 6

La chaîne de suspicion a (enfin) pu être implémenté. Elle a nécessité l'implémentation d'une - vraie - timeline car celle proposée par la librairie Mesa-ABM ne correspondait à ce que je recherchait : chaque tour, les connections, ainsi que les civilisations mortes sont stockées dans deux dictionnaires séparés connection_logs et removed_agents qui ont pour clef (key) le tour en question et pour valeur (value) respectivement les connections et les agents morts ce tour. C'est deux dictionnaires (un peu lourd) nous permettent de revenir à n'importe qu'elle étapes de l'instance en cours du modèle. Grâce à cette nouvelle timeline, la notion de "temporalité" est intégrée au modèle. La chaîne de suspicion est fondée sur cette notion de temporalité car elle va venir retarder la mise ne contact des agents qui sont soumis à cette chaîne.

Les fonctions contact() et step() de la class CivModel ont été séparé avoir que l'utilisation de step soit plus cohérente et que le code soit plus clair.

In [ ]:
class CivModel(Model):

    def __init__(self, N):
        self.connection_logs = {}  # connections effectuées
        self.removed_agents = {}  # agents enlevés (list unique id)

        # Chaîne de suspicion
        self.suspicions = {} # key : (agent1, agent2), value : suspicion_cooldown

### Utilitaires ###############################################################

    def remove_store(self, agent_id):
        """We use this function instead of the mesa remove() implemented method. It allows storing after supression"""
        self.removed_agents[self.timeline] = []
        self.removed_agents[self.timeline].append(
            self.schedule._agents[agent_id])
        self.schedule.remove(self.schedule._agents[agent_id])

    def restitute_old_state(self, time):
        """Permet de revenir à l'état du model à l'étape 'time'"""
        old_state = self.schedule._agents
        to_add = [v for k, v in self.removed_agents.items() if k >= time]

        # A remplacer
        n = []
        for i in to_add:
            n += i
        to_add = n

        # print(to_add)
        for agent in to_add:
            old_state[agent.unique_id] = agent
        # print(old_state.keys())

        return old_state

###############################################################################
    
### Logique ###################################################################
    
    def contact(self, a, b):
        """Logique de contact entre deux agents a et b, soit deux agents, cette fonction compare leur
            self.type (0 pour Pacifique (P) et 1 pour aggressif (A)) et leur niveau technologique
            (self.tech_lvl) si besoin"""

        if a in list(self.schedule._agents) and b in list(self.schedule._agents):
            agent_a = self.schedule._agents[a]
            agent_b = self.schedule._agents[b]

            if a == b:
                print("Meme agent")
                return
            # Si A ne detect pas B, vérifie que B ne detect pas A puis passe
            elif not self.detect(agent_a, agent_b):
                print("Agent", a, "ne rentre pas en contact avec Agent",
                      b, ": AR", agent_a.reception, "/ BE", agent_b.emission)
                return

                if not self.detect(agent_b, agent_a):
                    print("Agent", b, "ne rentre pas en contact avec Agent",
                          a, ": BR", agent_b.reception, "/ AE", agent_a.emission)
                    return
                else:
                    print("Mais Agent", b, "rentre en contact avec Agent",
                          a, ": BR", agent_b.reception, "/ AE", agent_a.emission)
        else:
            return

        print("Agents", a, b, "entre en contact !",)
        if agent_a.type < agent_b.type:  # Si b Aggressif et a Pacifist
            self.remove_store(a)
            # self.schedule.remove(agent_a)  # remove a
            print("Agent", a, "destroyed by Agent", b)
        elif agent_a.type > agent_b.type:  # Si b Pacifist et a Aggressif
            self.remove_store(b)
            # self.schedule.remove(agent_b)  # remove b
            print("Agent", b, "destroyed by Agent", a)
        elif agent_a.type == 1 and agent_b.type == 1:  # Si b Aggressif et a Aggressif, regarde le tech_lvl
            print("Agents", a, "and", b, "are both aggresive")
            # Niv technologique B == Niv technologique A <=> b Pacifist et a Pacifist, donc passe
            if agent_a.tech_lvl == agent_b.tech_lvl:
                print(a, "and", b,
                      "have the same technological level, nothing appends")
                return
            elif agent_a.tech_lvl < agent_b.tech_lvl:  # Si B a un meilleur NT que A, B détruit A
                self.remove_store(a)
                # self.schedule.remove(agent_a)
                print("Agent", b, "stronger than Agent", a)
                print("Agent", a, "destroyed by Agent", b)
            else:
                self.remove_store(b)
                # self.schedule.remove(agent_b)  # Same mais inverse
                print("Agent", a, "stronger than Agent", b)
                print("Agent", b, "destroyed by Agent", a)
        else:
            # Cas où les deux agents sont pacifistes et la chaine de suspicion s'engrange, à implémenter
            print("Agent", a, "and", b,
                  "are both pacifists -> chaine de suspicion (implémenté)----------------------")
            if (a,b) not in self.suspicions:
                self.suspicions[(a,b)] = suspicion_cooldown
                print(self.suspicions)
            # Niv technologique B == Niv technologique A <=> b Pacifist et a Pacifist, donc passe
            if agent_a.tech_lvl == agent_b.tech_lvl:
                print(a, "and", b,
                      "have the same technocontactal level, nothing appends")
                return
            elif agent_a.tech_lvl < agent_b.tech_lvl:  # Si B a un meilleur NT que A, B détruit A
                self.remove_store(a)
                # self.schedule.remove(agent_a)
                print("Agent", b, "stronger than Agent", a)
                print("Agent", a, "destroyed by Agent", b)
            else:
                self.remove_store(b)
                # self.schedule.remove(agent_b)  # Same mais inverse
                print("Agent", a, "stronger than Agent", b)
                print("Agent", b, "destroyed by Agent", a)

    def step(self):
        """Agissement du model à chaque étape"""

        # Compte le nombre de tour
        print("Tour :", self.timeline)
        self.timeline += 1

        # Connecte les agents
        r_l = self.random_connect()
        # Add the ongoing connection to the log (so it can be reused afterwards)
        self.connection_logs[self.timeline] = r_l
        print(r_l)

        #Logique de contact et de 
        self.resolve_suspicious()
        for a, b in r_l:
            self.contact(a, b)

        # Renvoie la liste des agents ayant survécu
        print("Survivors:")
        # affiche les agents restant
        self.schedule.step()
        print("Nombre d'agents restant", len(self.schedule._agents))

    def resolve_suspicious(self):
        for agentA, agentB in self.suspicions.keys():
            if self.suspicions[(agentA, agentB)] == 0:
                self.contact(agentA, agentB)
                print("Argh, ils ont craqué !", agentA, agentB)
                continue
            self.suspicions[(agentA, agentB)] -= 1
            print("Les agents", agentA, "et", agentB, "se méfient")
        print(self.suspicions)

        # delete toutes les chaines terminées
        to_del = list(self.suspicions.items())
        for k, v in to_del:
            if v <= 0:
                del self.suspicions[k]
        print(self.suspicions)


###############################################################################

#### Version 7

Une petite interface tkinter est mise au point, permettant de manipuler le modèle plus facilement qu'en fouillant dans la console remplie de print(), elle permet de voyager parmi les différentes étapes de l'instane en cours du modèle et de se rendre compte de la progression de celui, cette fonctionnalité est possible grâce à la fonction restitute_state() et à l'ajout d'un historique qui relie les deux dictionnaires removed_agents et connection_logs.

La fonction restitute_state() a été modifiée et renommer afin de pouvoir l'intégrer à l'interface graphique.

Dans cette interface, on peut aussi voir l'état actuel de l'univers en 3D ! (bon ça rame à mort mais c'est chouette non ?)

Des indicateurs, visibles dans l'interface ont aussi été ajoutés.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)

class CivModel(Model):

    def __init__(self, N):
        # Historique
        self.historique = {} # dict[tour[int], list[agents]]
        self.historique[0] = list(self.schedule._agents.values())

### Utilitaires ###############################################################

    def restitute_agents(self, time):
        """Permet de revenir à l'état du model à l'étape 'time'"""
        old_state = self.schedule._agents
        to_add = [v for k, v in self.removed_agents.items() if k >= time]

        # list[list]->list[int]
        n = []
        for i in to_add:
            n += i
        to_add = n

        # print(to_add)
        for agent in to_add:
            old_state[agent.unique_id] = agent
        # print(old_state.keys())

        return old_state

###############################################################################

### Indicateurs ###############################################################
    
    def distance_moyenne(self, time):
        agent_ids = self.historique[time]
        sum_dist = 0
        n = 0
        for a, b in self.distances_log.key():
            if a in agent_ids and b in agent_ids:
                sum_dist += self.distances_log[(a, b)]
                n +=1

        return sum_dist/n

    def ratio_P(self, time):
        agent_ids = self.historique[time]
        tot_P = 0
        for i in agent_ids:
            if self.schedule._agents[i] == 0:
                tot_P += 1

        return tot_P/len(agent_ids)

    def ratio_A(self, time):
        return math.abs(1-self.ratio_P(time))

    def ems_moyen(self, time):
        agent_ids = self.historique[time]
        sum_ems = 0
        n = 0
        for i in agent_ids:
            sum_ems += self.schedule._agents[i].emission
            n += 1

        return sum_ems/n
        
    def rec_moyen(self, time):
        agent_ids = self.historique[time]
        sum_ems = 0
        n = 0
        for i in agent_ids:
            sum_ems += self.schedule._agents[i].reception
            n += 1

        return sum_ems/n

    def tech_moyen(self, time):
        agent_ids = self.historique[time]
        sum_ems = 0
        n = 0
        for i in agent_ids:
            sum_ems += self.schedule._agents[i].tech_lvl
            n += 1

        return sum_ems/n

    def nb_contact(self, time):
        return

###############################################################################



def main():

    model = CivModel(E)

    ### MAIN LOGIC ####
        
    for i in range(I):
        model.step()

    ###################

    ##### TKINTER PARAMETERS & INITIALIZATION #####

    root = Tk()
    root.title("The Dark Forest Theory - Interface")
    root.geometry("1000x500")
    root.configure(bg='white')

    # Functions
    # slider func
    def update(s):
        
        to_plot = model.historique[slider.get()]
        #print("Historique à time=", slider.get(), [i.unique_id for i in to_plot])
        ax.clear()
        ax.axis('off')

        for a in to_plot:
            ax.scatter(a.x, a.y, a.z, color='b')

        plot.draw()  
        
        # Labels update
        nombre_agents.set("Nombre de civilisation "+str(len(to_plot)))


    # Frames
    plot_frame = Frame(root, bg='white')
    button_frame = Frame(root, bg='white')

    # Widgets
    quitter = Button(button_frame, bg='white', text='Quitter', command=root.quit)
    slider = Scale(root, bg='white', from_=0, to=I-1, orient=HORIZONTAL, state='active', tickinterval=round(I/4), command=update)

    # Labels
    nb_init_agents = Label(button_frame, bg='white', padx=10, text="Nombre initial de civilisation "+ str(model.nb_agents))
    nombre_agents = StringVar()
    nombre_agents.set("ahah")
    nb_agents = Label(button_frame, bg='white', padx=10, textvariable=nombre_agents)

    # Grid configuration
    root.columnconfigure(0, weight=1)
    root.rowconfigure(0, weight=1)

    # Grid layout
    # Frames
    plot_frame.grid(row=0, column=0, rowspan=3, sticky='nsew')
    button_frame.grid(row=0, column=1, rowspan=3, sticky='nsew')
    # Widgets
    quitter.grid(row=0, column=0, sticky='nsew')
    slider.grid(row=3, column=0, columnspan=2, sticky='nsew')
    # Labels
    nb_init_agents.grid(row=1, column=0, sticky='ne')
    nb_agents.grid(row=2, column=0, sticky='ne')

    # 3D graph
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.axis('off')
    plot = FigureCanvasTkAgg(fig, master=plot_frame)
    plot.get_tk_widget().pack()
    update('s')

    ###############################################

    root.mainloop()

#### Version 8 

Version final de notre projet, elle est disponible sur le github est possède toutes les caractéristiques des versions précédentes auquelles on été rajouté du contenu (indicateurs, plots etc...)